# 손글씨이미지데이터(MNIST)를 이용한 예측시스템

- MNIST(바이너리 데이터)
- 바이너리 데이터를 디코딩하여(디코딩 처리법)
- 샘플 데이터(전체데이터의 일부분)을 이용하여 머신러닝 모델을 구축
- 예측 수행 (64%)
- 머신러닝 시험 테스트 => 정확도를 높이는 것(94%)

# 연구목표

- 손글씨 이미지(0-9)를 학습시켜서, 새로운 손글씨 이미지가 들어오면 0-9중 어떤 것인지 예측한다.
- 원본 데이터를 디코딩하여(바이너리 데이터) 개별 이미지로 저장
- 픽셀 데이터를 벡터화 처리
- 정확도를 94% 이상 높이는 것

# 데이터 획득/수집

- http://yann.lecun.com/exdb/mnist/
- 1. 사이트를 긁어서,
- 2. 데이터의 URL을 직접 획득하여,
- 3. 로컬에 바로 저장 (URL을 직접 연결하여 로컬PC에 파일로 원하는 위치에 저장)

In [0]:
import urllib.request as req
from bs4 import BeautifulSoup

In [0]:
target_url = 'http://yann.lecun.com/exdb/mnist/'
# 요청
res= req.urlopen(target_url)
# 파싱
soup = BeautifulSoup(res, 'html5lib')

In [0]:
# URL이 <tt> 태그 안에 존재 => 4개까지 가져오자
tts = soup.find_all('tt')[:4]

In [0]:
files = list()

for tt in tts:
  # print(tt.a.string)
  # print(tt.find('a').string)
  file_url = target_url + tt.a.string
  files.append(file_url)

In [0]:
# 다음 단계를 위해서 최종 URL이 담긴 리스트로 전달하겠다
# files 
files = [ tt.a.string for tt in tts]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

# 데이터 준비

- 압축 해제
- 디코딩
- 벡터화

In [0]:
import os, os.path, gzip

In [0]:
# 파일 저장 위치 지정
save_path = '/content/drive/My Drive/Colab File/exam_MNIST'
# save_path = './data/mnist'

In [0]:
from tqdm import tqdm_notebook

# 파일 저장 위치 지정
# save_path = '/content/drive/My Drive/Colab File/exam_MNIST'
# save_path = './data/mnist'
# 해당 디렉토리가 없다면 
if not os.path.exists( save_path ):
  # 디렉토리 생성해라
  os.makedirs(save_path)

# 저장
# for file in files:
for file in tqdm_notebook(files):
  # 로컬에 저장 위치
  local_path = '%s/%s'%(save_path, file)
  if not os.path.exists(local_path):
    req.urlretrieve(target_url + file ,local_path)


In [0]:
# 압축해제
for file in tqdm_notebook(files):
  # 원본 파일 (*.gz)
  ori_path = '%s/%s'%(save_path, file)
  # 압축해제 파일 (*) <= .gz 제거
  raw_path = '%s/%s'%(save_path, file[:-3])
  # 파일 오픈
  with gzip.open(ori_path, 'rb') as fg:
    # 대용량이면 분할해서, 소용량이니까 그냥 전체 읽기
    tmp = fg.read()
    with open(raw_path, 'wb') as f:
      f.write(tmp)


## 파일의 구조

## label 파일의 구조
- 구조
  > magic number : 4byte  
  > label 개수 : 4byte  
  > label : 나머지~(1byte씩)  

## image 파일의 구조
- 구조
  > magic number : 4byte  
  > image 개수 : 4byte  
  > 이미지 가로 길이 : 4byte  
  > 이미지 세로 길이 : 4byte  
  > 이미지 : 28*28(784byte) byte가 image 개수만큼 반복  

## 정수(integer) 처리 
- 정수값을 읽을 때, 앞에서부터 읽을 것인가, 뒤에서부터 읽을 것인가.
- 오더링(ordering)
  - 빅 엔디언
  - 리틀 엔디언

### 빅 엔디언

- 값 : 0x12345678

|메모리주소|0x100|0x101|0x102|0x103|
|:--:|:--:|:--:|:--:|:--:|
|값|0x12|0x34|0x56|0x78|

### 리틀 엔디언

- 값 : 0x12345678

|메모리주소|0x100|0x101|0x102|0x103|
|:--:|:--:|:--:|:--:|:--:|
|값|0x78|0x56|0x34|0x12|


## 규격에 의해서 본 파일들은
- ***high endian(빅 엔디언)***으로 저장되어있다

In [0]:
# 디코딩 시에는 파일의 바이트계산을 요구한다.
# 데이터중 레이블의 크기
# 4 + 4 + 60000(byte)
# 4 + 4 + 4 + 4 + 60000*28*28(byte)

In [0]:
# 바이너리 데이터를 읽을 때 사용하는 모듈
import struct

In [0]:
# 함수로 파일을 풀어보겠다!
def decode_mnist(dataType="train", dir='/content/drive/My Drive/Colab File/exam_MNIST'):
  # 레이블 파일명 생성 : train-labels-idx1-ubyte
  label_name = f'{dir}/{dataType}-labels-idx1-ubyte'
  # 이미지 파일명 생성 : train-images-idx3-ubyte
  image_name = f'{dir}/{dataType}-images-idx3-ubyte'
  print (label_name, '\n', image_name)
  # 파일 오픈
  label_f = open(label_name, 'rb')
  image_f = open(image_name, 'rb')
  # ---------------------------------------------
  # 바이너리 데이터를 읽어서 csv에 이미지 데이터를 저장하겠다.
  # 훈련용(60000, 784+1) or 테스트용(10000, 784+1)
  # 이미지는 28x28 픽셀 => 784 픽셀 => 컬러(grayscale, RGB)
  # grayscale : 0-255 => 255 = 0xFF
  # 2^8 = 256 => 0-255 => 1Byte => 8bit
  # csv에 저장한다 -> 바이너리를 디코딩해서 csv에 데이터로 적재하겠다!
  csv_f = open( f'{dir}/{dataType}.csv', 'w', encoding='utf-8' )
  # ---------------------------------------------
  # 바이너리 파일을 해독(디코딩)
  # 1. 헤더를 읽는다.
  # 1-1 레이블의 헤더 : 4+4 Bytes
  LABEL_HEAD_SIZE = 4 + 4
  # 빅 엔디안 : >
  # 2번 읽을 것이다. 읽은 결과가 2개이다: II(알파벳 i 대분자)
  # 레이블 바이너리 파일에서 8 Byte만 읽어서 2개로 추출하라.
  # 단 데이터는 빅 엔디안으로 존재한다.
  magic, label_cnt = struct.unpack(">II", label_f.read(LABEL_HEAD_SIZE))
  print (magic, label_cnt)
  # 이미지의 헤더를 읽는다.
  IMAGE_HEAD_SIZE = 4 + 4 + 4 + 4
  magic_img, image_cnt, col, row = struct.unpack(">IIII", image_f.read(IMAGE_HEAD_SIZE))
  print(magic_img, image_cnt, col, row)

  pixels = row*col

  # ---------------------------------------------
  # 파일 닫기
  # IO는 안전하게 => defense code
  if label_f: label_f.close()
  if image_f: image_f.close()
  if csv_f: csv_f.close()

decode_mnist() 

/content/drive/My Drive/Colab File/exam_MNIST/train-labels-idx1-ubyte 
 /content/drive/My Drive/Colab File/exam_MNIST/train-images-idx3-ubyte
2049 60000
2051 60000 28 28


In [0]:
def decode_mnist(dataType="train", dir='/content/drive/My Drive/Colab File/exam_MNIST', samples=1000):
  label_name = f'{dir}/{dataType}-labels-idx1-ubyte'  
  image_name = f'{dir}/{dataType}-images-idx3-ubyte'
  label_f = open( label_name, 'rb')
  image_f = open( image_name, 'rb')
  csv_f = open( f'{dir}/{dataType}.csv', 'w', encoding='utf-8' )
  LABEL_HEAD_SIZE = 4 + 4
  magic, label_cnt = struct.unpack('>II', label_f.read(LABEL_HEAD_SIZE) )
  IMAGE_HEAD_SIZE = 4 + 4 + 4 + 4
  magic_img, image_cnt, row, col = struct.unpack('>IIII', image_f.read(IMAGE_HEAD_SIZE) )
  pixels = row * col
  # 데이터의 총개수만큼 반복해서 레이블,이미지를 읽어서 csv에 기록
  for idx in range(label_cnt):
    # 특정한 횟수만큼 데이터를 디코딩하여 csv에 기록했다면, 중단
    if idx >= samples:break
    # 레이블은 한번 반복할때마다 1 byte를 읽는다
    # unsigned byte => B
    label_value = struct.unpack( 'B', label_f.read(1) )    
    label = label_value[0] # (5,) => 0번째만 추출

    # 이미지는 한번 반복할때마다 784 byte를 읽는다 => pixels
    binary_data = image_f.read( pixels )
    #print( type(binary_data), len(binary_data))
    #byte 데이터를 하나씩 꺼내서 문자열로 변환하여 리스트로담는다
    strPixelData= list( map( lambda x:str(x) , binary_data ) ) 
    #print(strPixelData)
    # csv에 기록하기 -> 구분자 -> ,
    # 레이블,픽셀, 픽셀,..... 픽셀
    csv_f.write( str(label) + ',' )
    # strPixelData => ,를 구분자로 하여 한개의 문자열로 변환
    csv_f.write( ','.join(strPixelData) + '\n' )
    #break
    
    # 테스트로 binary_data를 하나 파일로 저장하여 확인
    # pgm 파일 포멧
    if idx == 0: # 1회만 수정
      with open(f'{dir}/example.pgm', 'w', encoding='utf-8') as f:
        f.write( 'P2 28 28 255\n' + ' '.join(strPixelData))
    

  if label_f:label_f.close()
  if image_f:image_f.close()
  if csv_f:csv_f.close()

In [0]:
# 최종 학습용 데이터 준비
decode_mnist(dataType='train', samples=750)
decode_mnist(dataType='t10k', samples=250)

# 최종산출물 => train.csv, t10k.csv

# 데이터 탐색/분석

- 픽셀 데이터의 정규화 추가
- 훈련에 필요한 데이터의 형태도 준비

In [0]:
# csv를 읽어서, 정규화 처리, 훈련할 수 있는 형태로 반환
# dataType : "train" or "t10k"
def load_csv(dataType="train", dir='/content/drive/My Drive/Colab File/exam_MNIST'):
  # 0. 데이터를 담을 자료구조
  labels = list()
  images = list()
  # 1. csv 파일 오픈
  with open( f'{dir}/{dataType}.csv', 'r' ) as f:
    # 한줄씩 읽는다.
    for line in f:
      tmp = line.split(',')
      # 정답 데이터 담기 => 타입은 수치로 변환
      labels.append( int(tmp[0]) )
      # 이미지 데이터 담기
      # 각 픽셀을 256개로 정규화하여서 리스트로 처리
      # image = list()
      # for i in tmp[1:]:
      #   image.append(int(i)/256)
      # images.append( image )
      images.append( list( map( lambda x:int(x)/256, tmp[1:] ) ) )

  return {"labels":labels , "images":images }

In [0]:
def load_csv(dataType="train", dir='/content/drive/My Drive/Colab File/exam_MNIST'):
  labels = list()
  images = list()
  with open( f'{dir}/{dataType}.csv', 'r' ) as f:
    for line in f:
      tmp = line.split(',')
      labels.append( int(tmp[0]) )
      images.append( list( map( lambda x:int(x)/256, tmp[1:] ) ) )

  return {"labels":labels , "images":images }

# 데이터 모델링

In [0]:
# 0. 모듈 가져오기
from sklearn import svm, model_selection, metrics

In [0]:
# 1. 알고리즘 생성
clf = svm.SVC()

In [0]:
# 최종 학습용 데이터 준비
decode_mnist(dataType='train', samples=750)
decode_mnist(dataType='t10k', samples=250)

# 최종산출물 => train.csv, t10k.csv

In [0]:
# 2. 훈련, 테스트 데이터 준비
train = load_csv()
test = load_csv(dataType='t10k')

In [0]:
# 3. 학습
clf.fit(train['images'], train['labels'])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
# 4. 예측
predict = clf.predict(test["images"])

In [0]:
# 5. 정확도 확인 : 성능 평가
metrics.accuracy_score( test['labels'], predict )

0.9

In [0]:
t = metrics.classification_report( test['labels'], predict )
print(t)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        34
           2       0.81      0.92      0.86        24
           3       1.00      0.70      0.82        23
           4       0.91      0.94      0.93        33
           5       0.72      0.84      0.78        25
           6       1.00      0.86      0.93        22
           7       0.81      0.90      0.85        29
           8       0.93      0.93      0.93        14
           9       0.92      0.89      0.91        27

    accuracy                           0.90       250
   macro avg       0.91      0.90      0.90       250
weighted avg       0.91      0.90      0.90       250



# 머신러닝 실기평가

- 95% 이상 정확도로 높이시오
  - 95%까지 실기점수 60점
  - 그 이상은 랭킹순으로 가산점 부여
  - 최종 주피터 파일 제출(구글 드라이브)
  - 이름_정확도.ipynb
  - 시작은 14:00

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
# 최종 학습용 데이터 준비
decode_mnist(dataType='train', samples=750)
decode_mnist(dataType='t10k', samples=250)

# 최종산출물 => train.csv, t10k.csv

# 2. 훈련, 테스트 데이터 준비
train = load_csv()
test = load_csv(dataType='t10k')

In [0]:
# SVC 알고리즘에 대한 파라미터 값 범주( 어디서부터 ~ 어디까지 ) 지정
# 기본값을 센터에 두고 전후를 나열
param_grid = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

# 교차 검증을 통해서 파라미터 최적화 값을 확인
# cv : 교차 검증수(폴드) 지정
# cv=5 => 폴드 수가 5개, 5세트를 지정한다, 이 중 한세트가 검증용 데이터
grid = GridSearchCV( SVC(), param_grid, cv=5 )

# 훈련
grid.fit(train['images'], train['labels'])

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
# 평가
print(grid.best_score_)
print(grid.best_params_)

# 0.9
# {'C': 10, 'gamma': 0.01}

0.892
{'C': 10, 'gamma': 0.01}


In [0]:
# 최종 학습용 데이터 준비
decode_mnist(dataType='train', samples=15000)
decode_mnist(dataType='t10k', samples=5000)

# 최종산출물 => train.csv, t10k.csv

# 2. 훈련, 테스트 데이터 준비
train = load_csv()
test = load_csv(dataType='t10k')

# 1. 알고리즘 생성
clf = SVC(C=10, gamma=0.01)

# 3. 학습
clf.fit(train['images'], train['labels'])

# 4. 예측
predict = clf.predict(test["images"])

# 5. 정확도 확인 : 성능 평가
print(metrics.accuracy_score( test['labels'], predict ))
t = metrics.classification_report( test['labels'], predict )
print(t)

0.9608
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       460
           1       0.96      0.99      0.98       571
           2       0.96      0.96      0.96       530
           3       0.95      0.97      0.96       500
           4       0.95      0.96      0.96       500
           5       0.98      0.95      0.96       456
           6       0.97      0.96      0.96       462
           7       0.96      0.94      0.95       512
           8       0.97      0.96      0.96       489
           9       0.95      0.94      0.94       520

    accuracy                           0.96      5000
   macro avg       0.96      0.96      0.96      5000
weighted avg       0.96      0.96      0.96      5000



In [0]:
# 최종 학습용 데이터 준비
decode_mnist(dataType='train', samples=30000)
decode_mnist(dataType='t10k', samples=10000)

# 최종산출물 => train.csv, t10k.csv

# 2. 훈련, 테스트 데이터 준비
train = load_csv()
test = load_csv(dataType='t10k')

# 1. 알고리즘 생성
clf = SVC(C=10, gamma=0.01)

# 3. 학습
clf.fit(train['images'], train['labels'])

# 4. 예측
predict = clf.predict(test["images"])

# 5. 정확도 확인 : 성능 평가
print(metrics.accuracy_score( test['labels'], predict ))
t = metrics.classification_report( test['labels'], predict )
print(t)

0.979
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.97      0.98      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.98      0.97      0.97       892
           6       0.98      0.98      0.98       958
           7       0.97      0.97      0.97      1028
           8       0.98      0.97      0.98       974
           9       0.98      0.97      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [185]:
# 최종 학습용 데이터 준비
decode_mnist(dataType='train', samples=60000)
decode_mnist(dataType='t10k', samples=10000)

# 최종산출물 => train.csv, t10k.csv

# 2. 훈련, 테스트 데이터 준비
train = load_csv()
test = load_csv(dataType='t10k')

# 1. 알고리즘 생성
clf = SVC(C=10, gamma=0.01)

# 3. 학습
clf.fit(train['images'], train['labels'])

# 4. 예측
predict = clf.predict(test["images"])

# 5. 정확도 확인 : 성능 평가
print(metrics.accuracy_score( test['labels'], predict ))
t = metrics.classification_report( test['labels'], predict )
print(t)

0.9833
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.98      0.98      0.98       892
           6       0.99      0.99      0.99       958
           7       0.98      0.98      0.98      1028
           8       0.98      0.98      0.98       974
           9       0.97      0.97      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



# 시스템통합 (생략)